In [10]:
import keras
import pandas
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.optimizers import SGD
import numpy as np
from time import time
from collections import Counter
import datetime
from keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint

In [233]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]
kdd_data = pandas.read_csv("data\\dataset2", header=None, names = col_names)
kdd_data.describe()


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,147838.000000,1.478380e+05,1.478380e+05,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,...,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000
mean,123.138990,5.232784e+04,6.706823e+03,0.000467,0.045205,0.001339,0.756152,0.019312,0.360232,0.119509,...,195.740101,152.716372,0.731881,0.103555,0.295514,0.043844,0.115120,0.113749,0.095216,0.092336
std,1563.874919,4.057662e+06,1.649974e+05,0.021599,0.350530,0.041915,3.870293,0.144336,0.480069,1.240077,...,101.219335,114.850269,0.413515,0.269012,0.439489,0.149056,0.307468,0.315510,0.267315,0.281674
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.100000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,155.000000,13.000000,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.050000e+02,4.300000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,250.000000,1.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,3.340000e+02,1.740000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,255.000000,255.000000,1.000000,0.030000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,58329.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,5.000000,1.000000,238.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [234]:
train_labels = np.array(kdd_data['label'])
kdd_data = kdd_data.drop(['protocol_type','service','flag','num_outbound_cmds','is_host_login','label','num_compromised',
                          'su_attempted','num_root','rerror_rate','diff_srv_rate','srv_diff_host_rate','dst_host_diff_srv_rate',
                          'dst_host_srv_diff_host_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate'], axis=1)



In [235]:
#print(kdd_data.count)
kdd_data=kdd_data/kdd_data.max()
train_data = kdd_data.values
train_data = train_data[:,:]

#print(kdd_data.count)


x = list(set(train_labels))

In [236]:
#converting labels into 5 types
train_label2 = []
for label in train_labels:
    if label in ['nmap.', 'portsweep.', 'ipsweep.', 'satan.','mscan.','saint.']:
        label = 'probing'
    elif label in ['land.','pod.','teardrop.','back.','neptune.','smurf.','apache2.','mailbomb.','processtable.','udpstorm.']:
        label = 'dos'
    elif label in ['spy.', 'phf.', 'multihop.', 'ftp_write.', 'imap.', 'warezmaster.', 'warezclient.', 'guess_passwd.','sendmail.','named.','snmpgetattack.','snmpguess.','xlock.','xsnoop.','worm.']:
        label = 'r2l'
    elif label in ['buffer_overflow.', 'rootkit.','loadmodule.','perl.','httptunnel.','ps.','sqlattack.','xterm.']:
        label = 'u2r'
    train_label2.append(label)

In [237]:
#converting labels into int format
y = list(set(train_label2))
dictY = {}
for i, label in enumerate(y):
    dictY.update({label:i})
print(dictY)
train_label3 = []
for label in train_label2:
    train_label3.append(dictY[label])
print(len(train_label3))
#converting int labels into one hot encoding
one_hot_labels = keras.utils.to_categorical(train_label3, num_classes=5)
print(one_hot_labels)

{'normal.': 0, 'u2r': 1, 'r2l': 2, 'probing': 3, 'dos': 4}
147838
[[0.000000 0.000000 0.000000 0.000000 1.000000]
 [0.000000 0.000000 1.000000 0.000000 0.000000]
 [0.000000 0.000000 1.000000 0.000000 0.000000]
 ..., 
 [0.000000 0.000000 1.000000 0.000000 0.000000]
 [0.000000 0.000000 1.000000 0.000000 0.000000]
 [0.000000 1.000000 0.000000 0.000000 0.000000]]


In [238]:
model = Sequential()
model.add(Dense(12, input_shape= (train_data.shape[1],), activation='relu'))
model.add(BatchNormalization())
#model.add(Dropout(0.5, name='dropout_1'))
model.add(Dense(5, activation='softmax'))

sgd = SGD(lr=1e-4, decay=1e-5, momentum=0.9, nesterov=True,clipnorm=0.1)
model.compile(optimizer=sgd,
              loss='categorical_crossentropy', metrics=['accuracy'])

#model.load_weights('model.h5')
csv_logger = CSVLogger("training1.csv")
checkpoint = ModelCheckpoint("model.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
model.fit(train_data, one_hot_labels, batch_size = 100, epochs = 20, verbose = 1, validation_split = 0.2, callbacks = [csv_logger, checkpoint])

Train on 118270 samples, validate on 29568 samples
Epoch 1/20
118270/118270 [==============================] - 4s - loss: 1.6106 - acc: 0.3025 - val_loss: 1.5319 - val_acc: 0.3227
Epoch 2/20
118270/118270 [==============================] - 2s - loss: 1.4528 - acc: 0.3311 - val_loss: 1.3749 - val_acc: 0.3314
Epoch 3/20
118270/118270 [==============================] - 3s - loss: 1.2991 - acc: 0.3613 - val_loss: 1.2454 - val_acc: 0.3585
Epoch 4/20
118270/118270 [==============================] - 3s - loss: 1.1955 - acc: 0.5409 - val_loss: 1.1501 - val_acc: 0.6445
Epoch 5/20
118270/118270 [==============================] - 3s - loss: 1.1118 - acc: 0.6542 - val_loss: 1.0718 - val_acc: 0.6632
Epoch 6/20
118270/118270 [==============================] - 3s - loss: 1.0419 - acc: 0.6687 - val_loss: 1.0079 - val_acc: 0.6801
Epoch 7/20
118270/118270 [==============================] - 3s - loss: 0.9837 - acc: 0.6885 - val_loss: 0.9532 - val_acc: 0.7070
Epoch 8/20
118270/118270 [====================

In [239]:

model.load_weights('model.h5')
x = model.predict_classes(train_data)
print(x)
j = 0
for i, d in enumerate(x):
    if d == train_label3[i]:
        j = j+1
print(j/len(x))

147296/147838 [============================>.] - ETA: 0s[4 2 2 ..., 2 2 2]
0.8045631028558287


In [240]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]
kdd_data2 = pandas.read_csv("corrected\\corrected", header=None, names = col_names)
kdd_data2.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,311029.000000,3.110290e+05,3.110290e+05,311029.000000,311029.000000,311029.000000,311029.000000,311029.000000,311029.000000,311029.000000,...,311029.000000,311029.000000,311029.000000,311029.000000,311029.000000,311029.000000,311029.000000,311029.000000,311029.000000,311029.000000
mean,17.902736,1.731702e+03,7.479937e+02,0.000029,0.000762,0.000051,0.014677,0.002363,0.172476,0.011243,...,235.282681,199.193914,0.793494,0.024953,0.547919,0.004566,0.058764,0.058791,0.142659,0.141693
std,407.644400,1.276567e+05,1.612018e+04,0.005379,0.040367,0.009821,0.312068,0.049990,0.377794,1.958325,...,60.913298,100.306470,0.387090,0.096003,0.491963,0.035773,0.231296,0.232997,0.344380,0.346573
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.050000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,244.000000,0.970000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,5.200000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.032000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.010000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,57715.000000,6.282565e+07,5.203179e+06,1.000000,3.000000,3.000000,101.000000,4.000000,1.000000,796.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [241]:
test_labels = np.array(kdd_data2['label'])
kdd_data2 = kdd_data2.drop(['protocol_type','service','flag','num_outbound_cmds','is_host_login','label','num_compromised',
                            'su_attempted','num_root','rerror_rate','diff_srv_rate','srv_diff_host_rate',
                            'dst_host_diff_srv_rate','dst_host_srv_diff_host_rate','dst_host_rerror_rate',
                            'dst_host_srv_rerror_rate'], axis=1)

In [242]:
kdd_data2=kdd_data2/kdd_data2.max()
test_data = kdd_data2.values
test_data = test_data[:,:]


#converting labels into 5 types
test_label2 = []
for label in test_labels:
    if label in ['nmap.', 'portsweep.', 'ipsweep.', 'satan.','mscan.','saint.']:
        label = 'probing'
    elif label in ['land.','pod.','teardrop.','back.','neptune.','smurf.','apache2.','mailbomb.','processtable.','udpstorm.']:
        label = 'dos'
    elif label in ['spy.', 'phf.', 'multihop.', 'ftp_write.', 'imap.', 'warezmaster.', 'warezclient.', 'guess_passwd.','sendmail.','named.','snmpgetattack.','snmpguess.','xlock.','xsnoop.','worm.']:
        label = 'r2l'
    elif label in ['buffer_overflow.', 'rootkit.','loadmodule.','perl.','httptunnel.','ps.','sqlattack.','xterm.']:
        label = 'u2r'
    test_label2.append(label)

y2 = list(set(test_label2))
test_label3 = []
for label in test_label2:
    if label in dictY.keys():
        test_label3.append(dictY[label])
print(set(test_label3))

model.load_weights('model.h5')
x = model.predict_classes(test_data)
print(x)
j = 0
for i, d in enumerate(test_label3):
    if d == x[i] :
        j = j+1
print(j/len(test_label3))


{0, 1, 2, 3, 4}
310176/311029 [============================>.] - ETA: 0s[2 2 2 ..., 2 2 2]
0.7204890862266863


In [243]:
j=0
typeA=0
typeB=0
typeC=0
typeD=0
typeE=0
for i, d in enumerate(test_label3):
    if d == x[i] :
        if d==0:
            typeA = typeA+1
        elif d==1:
            typeB = typeB+1
        elif d==2:
            typeC = typeC+1
        elif d==3:
            typeD = typeD+1
        elif d==4:
            typeE = typeE+1
        j = j+1
print(j/len(test_label3))
print(typeA,typeB,typeC,typeD,typeE)

0.7204890862266863
22367 41 15935 3652 182098


In [244]:
i=(1)
q=Counter(x)
print(q)

Counter({4: 183058, 2: 57710, 3: 46185, 0: 22458, 1: 1618})


In [245]:
from sklearn.metrics import confusion_matrix


con=confusion_matrix(x,test_label3)
print(con)

[[ 22367      4     87      0      0]
 [  1381     41     88      5    103]
 [ 34674     41  15935    462   6598]
 [  1284    139     56   3652  41054]
 [   887      3     23     47 182098]]


In [246]:
par=np.zeros((4,5))

for i in range (0,5):
    par[0,i]=con[i,i]
    
for i in range (0,5):
    for j in range (0,5):
        if i!=j:
            par[1,i]=par[1,i]+con[i,j]

for i in range (0,5):
    for j in range (0,5):
        if i!=j:
            par[2,i]=par[2,i]+con[j,i]        

for i in range (0,5):
    for j in range (0,5):
        for k in range (0,5):
            if j!=i and k!=i:
                par[3,i]=par[3,i]+con[i,j]

                
#2D matrix par
#tp-0
#fn-1
#fp-2
#tn-3
#2D matrix ratio
#tpr-4
#fpr-5
#precision-6
#accuracy-7
ratio=np.zeros((4,5))
for i in range (0,5):
    ratio[0,i]=(par[0,i]/(par[0,i]+par[1,i]))*100
    
for i in range (0,5):
    ratio[1,i]=(par[2,i]/(par[2,i]+par[3,i]))*100
for i in range (0,5):
    ratio[2,i]=par[0,i]/(par[0,i]+par[2,i])
for i in range (0,5):
    ratio[3,i]=((par[0,i]+par[3,i])/(par[0,i]+par[1,i]+par[2,i]+par[3,i]))*100


np.set_printoptions(formatter={'float_kind':'{:f}'.format})       
print('Parameters')
print(par.astype(int))
print('Parameter ratio')
print(ratio)

Parameters
[[ 22367     41  15935   3652 182098]
 [    91   1577  41775  42533    960]
 [ 38226    187    254    514  47755]
 [   364   6308 167100 170132   3840]]
Parameter ratio
[[99.594799 2.533993 27.612199 7.907329 99.475576]
 [99.056750 2.879138 0.151774 0.301208 92.557418]
 [0.369135 0.179825 0.984310 0.876620 0.792237]
 [37.234635 78.257118 81.325756 80.147211 79.239558]]


VALUES
normal=2
u2r=1
r2l=0
probe=3
dos=4